In [10]:
import pandas as pd

In [11]:
data = pd.read_csv("adult.data.csv")
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [12]:
data["sex"].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

In [13]:
data_male = data[data["sex"] == "Male"]
data_male["age"].mean()

39.43354749885268

In [14]:
data['native-country'].value_counts().loc['United-States'] / len(data) * 100

89.5857006848684

In [15]:
def func():
    dm = data.groupby('salary')[['age']].mean()
    ds = data.groupby('salary')[['age']].std()
 
    d = pd.concat([dm,ds],axis=1)
    d.columns = ['mean','std']
    print(d)
func()

             mean        std
salary                      
<=50K   36.783738  14.020088
>50K    44.249841  10.519028


In [16]:
data[data['salary'] == '>50K']['education'].unique()

array(['HS-grad', 'Masters', 'Bachelors', 'Some-college', 'Assoc-voc',
       'Doctorate', 'Prof-school', 'Assoc-acdm', '7th-8th', '12th',
       '10th', '11th', '9th', '5th-6th', '1st-4th'], dtype=object)

In [17]:
for (race, sex), i in pd.groupby(data, ['race', 'sex']):
    print("Раса: {0}, пол: {1}".format(race, sex))
    print(i['age'].describe())

AttributeError: module 'pandas' has no attribute 'groupby'

In [18]:
status = [ i for i in data['marital-status'].unique() if i.startswith('Married') ]

married = data[(data['sex'] == 'Male') &
     (data['marital-status'].isin(status))]['salary'].value_counts()
print(married)
not_married = data[(data['sex'] == 'Male') &
     (~data['marital-status'].isin(status))]['salary'].value_counts()
print(not_married)
print("Женаты: {0}%".format(round(100 * married[1]/(married[0]+married[1]), 2)))
print("Холосты: {0}%".format(round(100 * not_married[1]/(not_married[0]+not_married[1]), 2)))

<=50K    7576
>50K     5965
Name: salary, dtype: int64
<=50K    7552
>50K      697
Name: salary, dtype: int64
Женаты: 44.05%
Холосты: 8.45%


In [19]:
max_work = data['hours-per-week'].max()
print("Максимальное число часов в неделю - {0} часов".format(max_work))

num_workers = data[data['hours-per-week'] == max_work].shape[0]
print("Всего людей с таким количеством часов - {0}".format(num_workers))

per_rich = data[(data['hours-per-week'] == max_work) & (data['salary'] == '>50K')].shape[0] / num_workers
print("Из них богатых - {0}%".format(round(100 * per_rich, 2)))

Максимальное число часов в неделю - 99 часов
Всего людей с таким количеством часов - 85
Из них богатых - 29.41%


In [20]:
for (country, salary), i in pd.groupby(data, ['native-country', 'salary']):
    print(country, salary, round(i['hours-per-week'].mean(), 2))

AttributeError: module 'pandas' has no attribute 'groupby'

In [21]:
pd.cut(data['age'], bins=3).value_counts()
cut_labels_3 = ['joung', 'adult', 'retiree']
cut_bins = [16, 35, 70, 100]

data['AgeGroup'] = pd.cut(data['age'],
                       bins=cut_bins,
                       labels=cut_labels_3)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary,AgeGroup
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,adult
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,adult
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,adult
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,adult
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,joung


In [22]:
more_50k = data[data["salary"] == ">50K"]

joung = more_50k[more_50k["AgeGroup"] == "joung"].shape[0]
print(f'Joung: {joung}')

adult = more_50k[more_50k["AgeGroup"] == "adult"].shape[0]
print(f'Adult: {adult}')

retiree = more_50k[more_50k["AgeGroup"] == "retiree"].shape[0]
print(f'Retiree: {retiree}')

print(f'Earns more 50k: adult - {adult}')

Joung: 1705
Adult: 6042
Retiree: 94
Earns more 50k: adult - 6042


In [23]:
occupation = data.groupby("occupation").size()
print(occupation)

occupation
?                    1843
Adm-clerical         3770
Armed-Forces            9
Craft-repair         4099
Exec-managerial      4066
Farming-fishing       994
Handlers-cleaners    1370
Machine-op-inspct    2002
Other-service        3295
Priv-house-serv       149
Prof-specialty       4140
Protective-serv       649
Sales                3650
Tech-support          928
Transport-moving     1597
dtype: int64


In [25]:
# filter_age = data.groupby("occupation").agg({"age":"mean"})
# f = filter_age[(filter_age['age'] <= 40)]
# print(f)

# filter_hours = data['hours-per-week'] > 5
# data.loc[filter_hours].groupby("occupation").size()

sort_data = data[(data['hours-per-week'] > 5) & (data["age"].mean() <= 40)]
data_filter = data["occupation"]
data_filter.head()

0         Adm-clerical
1      Exec-managerial
2    Handlers-cleaners
3    Handlers-cleaners
4       Prof-specialty
Name: occupation, dtype: object